<a href="https://colab.research.google.com/github/shielamms/Tensorflow-Notebooks/blob/main/CvD_Classifier_Transfer_Learning_with_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get the pre-trained model

This fetches the weights from the Inception V3 Model

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-03-28 10:47:31--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 142.251.163.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   135MB/s    in 0.6s    

2023-03-28 10:47:32 (135 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

In [ ]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape=(150,150,3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the pre-trained model
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
# pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')

print('Pre-trained model last layer output shape:', last_layer.output_shape) 

Pre-trained model last layer output shape: (None, 7, 7, 768)


In [ ]:
last_output = last_layer.output

## Build the rest of the model

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Flatten the output from the pre-trained model
x = layers.Flatten()(last_output)

# Add a Dense layer
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout to regularize the output
x = layers.Dropout(0.2)(x)

# Add the final layer
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 74, 74, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 74, 74, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                            

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Preprocessing

In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

--2023-03-28 11:10:52--  https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 172.253.63.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   135MB/s    in 0.5s    

2023-03-28 11:10:53 (135 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
zip_ref = zipfile.ZipFile('./cats_and_dogs_filtered.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
!ls /tmp/cats_and_dogs_filtered/

train  validation  vectorize.py


In [ ]:
!ls /tmp/cats_and_dogs_filtered/train

cats  dogs


In [ ]:
root_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(root_dir, 'train')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_dir = os.path.join(root_dir, 'validation')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=(1./255),
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=(1./255))

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
EPOCHS = 16

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2
)

Epoch 1/16
100/100 - 217s - loss: 1.0924 - accuracy: 0.8430 - val_loss: 0.2985 - val_accuracy: 0.8940 - 217s/epoch - 2s/step
Epoch 2/16
100/100 - 189s - loss: 0.2347 - accuracy: 0.9180 - val_loss: 0.1022 - val_accuracy: 0.9580 - 189s/epoch - 2s/step
Epoch 3/16
100/100 - 191s - loss: 0.2068 - accuracy: 0.9170 - val_loss: 0.0939 - val_accuracy: 0.9650 - 191s/epoch - 2s/step
Epoch 4/16
100/100 - 186s - loss: 0.1928 - accuracy: 0.9320 - val_loss: 0.0843 - val_accuracy: 0.9630 - 186s/epoch - 2s/step
Epoch 5/16


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()